In [154]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [155]:
data=pd.read_csv('../data/titanic_train.csv')
print(data.info())
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [156]:
# 选取数据集中有用的特征
data1 = data.drop(labels=['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
data1.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [157]:
data2 = data1.dropna()
print(data1.shape)
print(data2.shape)

(891, 8)
(712, 8)


In [158]:
data3=data2.replace({'Sex':{'male':1,'female':2},
                     'Embarked':{'S':1,'C':2,'Q':3}})
data3.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,1
1,1,1,2,38.0,1,0,71.2833,2
2,1,3,2,26.0,0,0,7.9250,1
3,1,1,2,35.0,1,0,53.1000,1
4,0,3,1,35.0,0,0,8.0500,1


In [159]:
x_train=data3.drop(labels='Survived',axis=1)
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,1
1,1,2,38.0,1,0,71.2833,2
2,3,2,26.0,0,0,7.9250,1
3,1,2,35.0,1,0,53.1000,1
4,3,1,35.0,0,0,8.0500,1


In [160]:
y_train=data3['Survived']
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [161]:
from sklearn.preprocessing import StandardScaler
S=StandardScaler()
S.fit(x_train)
x_train_stand=S.transform(x_train)

In [162]:
clf=SVC(kernel='linear',verbose=1)
clf.fit(x_train_stand,y_train)

[LibSVM]

SVC(kernel='linear', verbose=1)

In [163]:
scores = cross_val_score(clf, x_train_stand, y_train, cv=10)
scores

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

array([0.80555556, 0.76388889, 0.8028169 , 0.83098592, 0.74647887,
       0.78873239, 0.74647887, 0.73239437, 0.8028169 , 0.77464789])

In [164]:
test=pd.read_csv('../data/test.csv')
test1 = test.drop(labels=['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
test1.head()
test1.isnull().sum()
age_avg=int(x_train.mode()["Age"])
print(age_avg)
fare_avg=int(x_train.mode()["Fare"])
print(fare_avg)

24
13


In [165]:
test1['Age'] = test1['Age'].fillna(value = age_avg)
test1['Fare'] = test1['Fare'].fillna(value = fare_avg)
test1.isnull().sum()
test2=test1.replace({'Sex':{'male':1,'female':2},
                     'Embarked':{'S':1,'C':2,'Q':3}})
test2

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,3
1,3,2,47.0,1,0,7.0000,1
2,2,1,62.0,0,0,9.6875,3
3,3,1,27.0,0,0,8.6625,1
4,3,2,22.0,1,1,12.2875,1
...,...,...,...,...,...,...,...
413,3,1,24.0,0,0,8.0500,1
414,1,2,39.0,0,0,108.9000,2
415,3,1,38.5,0,0,7.2500,1
416,3,1,24.0,0,0,8.0500,1


In [166]:
S=StandardScaler()
S.fit(test2)
test2_stand=S.transform(test2)
prediction=clf.predict(test2_stand)
print(len(test2))
result=pd.DataFrame({'PassengerId':test2.index,'Survived':prediction.astype(np.int32)})

418


In [167]:
result["PassengerId"]=result["PassengerId"]+892
print(result)
result.to_csv('../data/result.csv',index=False)
val_data=pd.read_csv('../data/gender_submission.csv')
1-len(val_data.compare(result,keep_equal=False))/len(result)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


1.0